# Proceso de Autenticación

### Angel Alvarez

In [1]:
# Librerias Qiskit
from qiskit import *
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.extensions import *
# Plots
from qiskit.visualization import plot_histogram as plth
from random import choice

In [2]:
def GHZ_init(n):
    """
    Esta funcion genera una copuerta de n 
    qubits para transformar a estados GHZ.
    """
    qc = QuantumCircuit(n)

    #Estado GHZ
    qc.h(0)
    for i in range(1, n):
        qc.cx(0, i)
    
    GHZ = qc.to_gate()
    GHZ.name = '$GHZ$'
    
    return GHZ

def C_u(Jk):
    """
    Esta función toma un valor binario de 
    longitud 2  para realizar la codifi-
    cacion por transformacion C_{OP}.
    """
    qc = QuantumCircuit(2)
    
    if Jk == "0":
        qc.cx(0,1)
        name_ = "$C_0$"
    else :
        qc.h(0)
        qc.h(1)
        qc.cx(0,1)
        qc.h(0)
        qc.h(1)
        
        name_ = "$C_1$"
    
    c_gate = qc.to_gate()
    c_gate.name = name_
    
    return c_gate


In [3]:
def Authentication_Circuit(n_users, u_keys, qs_key):
    """
    Esta función crea el circuito de autenticación entre los usuarios
    y el QS. Devuelve el resultado de las medidas sobre las particulas n
    """
    n_qubits = 2*n_users + 1 ;

    qc = QuantumCircuit(n_qubits, n_users)
    #Estado GHZ
    
    GHZ = GHZ_init(n_users + 1)
    qc.append(GHZ, range(n_users+1))
    
    # Inicialización de los usuarios
    
    for i in range(n_users):
        if u_keys[i][i] == '1':
            qc.x(n_users + 1 + i)
    
    # Codificacion con C_op
    
    for i in range(n_users):
        j2i_u = u_keys[i][n_users]
        
        qc.append(C_u(j2i_u), [1+i,n_users+1+i])
    
    # Decodificaccion del QS
    
    j2i_qs = qs_key[n_users]
    c_op_qs = C_u(j2i_qs)
    
    for i in range(n_users):
        qc.append(c_op_qs , [0, n_users+1+i])
    
    qc.barrier()
    # Medida 
    qc.measure(range(n_users+1,n_qubits),range(n_users))

    # Corrida y visualización de los resultados
    sim = Aer.get_backend('qasm_simulator')
    results = execute(qc, backend = sim, shots = 1).result()
    counts = results.get_counts()
    RESULT = list(counts.keys())[0][::-1]
    
    return RESULT


In [4]:
def Users_Authenticated(res, qs_key):
    """
    Esta función confirma si el resultado del circuito 
    corresponde a lo esperado segun la llave del QS.
    """
    l = len(res)
    return ( res == qs_key[:l])

def Test_Authentication(n_users, n):
    """
    Esta función corre el proceso de certificado de la red de autenticación.
    """
    true = 0
    for i in range(n):
        common_key =  ''.join(choice('01') for _ in range(n_users+1))
        u_keys = [common_key for i in range(n_users)]
        qs_key = common_key
        res = Authentication_Circuit(n_users, u_keys, qs_key)
        
        if Users_Authenticated(res, qs_key):
            true = true +1
            
    print(f"Authentication runed: {n}\n")
    print(f"Number of users authenticated: {n_users}\n")
    print(f"Percentage of times all users got authenticated: {true/n*100}")
    

In [5]:
Test_Authentication(10, 100)

Authentication runed: 100

Number of users authenticated: 10

Percentage of times all users got authenticated: 100.0
